In [2]:
# Installing required packages
!pip install pyspark
!pip install findspark

In [1]:
import findspark
findspark.init()

In [2]:
# PySpark is the Spark API for Python. In this lab, we use PySpark to initialize the spark context. 
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, DoubleType, TimestampType
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
# Creating a spark context class
sc = SparkContext()

# Creating a spark session
spark = SparkSession \
    .builder \
    .appName("Prática da semana 2 - Aceleração Pyspark") \
    .config("spark.some.config.option", "some-value") \
    .config("spark.memory.offHeap.enabled","true") \
    .config("spark.memory.offHeap.size","10g")\
    .getOrCreate()

In [4]:
spark

# Airports

In [5]:
schema_air = StructType()\
    .add('faa', StringType(), True)\
    .add('name', StringType(), True)\
    .add('lat', DoubleType(), True)\
    .add('lon', DoubleType(), True)\
    .add('alt', DoubleType(), True)\
    .add('tz', IntegerType(), True)\
    .add('dst', StringType(), True)

airports = spark.read.format('csv')\
    .option('header', True)\
    .schema(schema_air)\
    .load('C:/Practice_Pyspark/airports.csv')

airports.printSchema()

root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: double (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



In [117]:
airports.toPandas()

,faa,name,lat,lon,alt,tz,dst,region,type,military,administration
0,04G,Lansdowne Airport,41.130472,-80.619583,1044.0,-5,A,MAINLAND-EAST,AP,False,NaN
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264.0,-5,A,MAINLAND-EAST,AP,False,NaN
2,06C,Schaumburg Regional,41.989341,-88.101243,801.0,-6,A,MAINLAND-EAST,NaN,False,NaN
3,06N,Randall Airport,41.431912,-74.391561,523.0,-5,A,MAINLAND-EAST,AP,False,NaN
4,09J,Jekyll Island Airport,31.074472,-81.427778,11.0,-4,A,MAINLAND-EAST,AP,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1392,ZUN,Black Rock,35.083228,-108.791778,6454.0,-7,A,MAINLAND-WEST,NaN,False,NaN
1393,ZVE,New Haven Rail Station,41.298669,-72.925992,7.0,-5,A,MAINLAND-EAST,AS,False,NaN
1394,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0.0,-5,A,MAINLAND-EAST,AS,False,NaN
1395,ZWU,Washington Union Station,38.897460,-77.006430,76.0,-5,A,MAINLAND-EAST,AS,False,NaN


### Pergunta 1

In [6]:
airports = airports.withColumn('alt',
                              F.when(F.col('alt') < 0, 0)    
                              .otherwise(F.col('alt')))

In [8]:
airports.select('alt').filter(F.col('alt') != 0).show()

+------+
|   alt|
+------+
|1044.0|
| 264.0|
| 801.0|
| 523.0|
|  11.0|
|1593.0|
| 730.0|
| 492.0|
|1000.0|
| 108.0|
| 409.0|
| 875.0|
|1003.0|
| 951.0|
|1789.0|
| 122.0|
| 152.0|
| 670.0|
|1134.0|
| 885.0|
+------+
only showing top 20 rows



### Pergunta 2

In [11]:
airports = airports.withColumn('dst',
                              F.when(
                                  F.col('tz').between(-7, -5), 'A')
                              .otherwise(F.col('dst')))

In [13]:
airports.filter((F.col('dst') == 'U')).show(5)

+---+--------------------+----------+-------------+------+---+---+
|faa|                name|       lat|          lon|   alt| tz|dst|
+---+--------------------+----------+-------------+------+---+---+
|19A|Jackson County Ai...|34.1758638|  -83.5615972| 951.0| -4|  U|
|BLD|Boulder City Muni...|   35.5651|     -114.514|2201.0| -8|  U|
|GCW|Grand Canyon West...|   35.5925|    -113.4859|4825.0| -8|  U|
|MXY|    McCarthy Airport|61.4370608|-142.90307372|1531.0| -8|  U|
|NGZ|         NAS Alameda|   37.7861|    -122.3186|  10.0| -9|  U|
+---+--------------------+----------+-------------+------+---+---+
only showing top 5 rows



### Pergunta 3

In [14]:
airports = airports.withColumn('dst',
                              F.when(F.col('dst') == 'U', 'A')
                              .otherwise(F.col('dst')))

In [15]:
airports.select('dst').filter(F.col('dst') == 'U').show()

+---+
|dst|
+---+
+---+



### Pergunta 4

In [16]:
airports = airports.withColumn('region',
                              F.when(F.col('lon') < -124, 'ALASKA')
                              .when(
                                  (F.col('lon') > -50) | (F.col('lat') < 24), 'OFFSHORE')
                              .when(F.col('lon').between(-124, -95), 'MAINLAND-WEST')
                              .when(
                                  (F.col('lon') > -95) & 
                                  (F.col('lon') <= -50), 'MAINLAND-EAST')
                              .otherwise(float('nan')))

In [17]:
airports.show(truncate = False)

+---+------------------------------------+----------------+-----------------+------+---+---+-------------+
|faa|name                                |lat             |lon              |alt   |tz |dst|region       |
+---+------------------------------------+----------------+-----------------+------+---+---+-------------+
|04G|Lansdowne Airport                   |41.1304722      |-80.6195833      |1044.0|-5 |A  |MAINLAND-EAST|
|06A|Moton Field Municipal Airport       |32.4605722      |-85.6800278      |264.0 |-5 |A  |MAINLAND-EAST|
|06C|Schaumburg Regional                 |41.9893408      |-88.1012428      |801.0 |-6 |A  |MAINLAND-EAST|
|06N|Randall Airport                     |41.431912       |-74.3915611      |523.0 |-5 |A  |MAINLAND-EAST|
|09J|Jekyll Island Airport               |31.0744722      |-81.4277778      |11.0  |-4 |A  |MAINLAND-EAST|
|0A9|Elizabethton Municipal Airport      |36.3712222      |-82.1734167      |1593.0|-4 |A  |MAINLAND-EAST|
|0G6|Williams County Airport         

### Pergunta 5

In [18]:
AP_CONDITION = ["Airport", "Tradeport", "Heliport", "Airpor", "Arpt"]

AP_REGEX = r'|'.join(map(lambda x : f".*({x}).*", AP_CONDITION))

In [19]:
AP_REGEX

'.*(Airport).*|.*(Tradeport).*|.*(Heliport).*|.*(Airpor).*|.*(Arpt).*'

In [20]:
airports = airports.withColumn('type', 
                              F.when(F.col('name').rlike(AP_REGEX), 'AP')
                              .when(F.col('name').contains('Aerodrome'), 'AD')
                              .when(F.col('name').rlike('.*Airpark.*|.*Aero Park.*'), 'AK')
                              .when(F.col('name').rlike('.*Station.*|.*Air Station.*'), 'AS')
                              .when(F.col('name').rlike('.*Field.*|.*Fld.*'), 'FL')
                              .otherwise(float('nan')))

In [21]:
airports.show(truncate = False)

+---+------------------------------------+----------------+-----------------+------+---+---+-------------+----+
|faa|name                                |lat             |lon              |alt   |tz |dst|region       |type|
+---+------------------------------------+----------------+-----------------+------+---+---+-------------+----+
|04G|Lansdowne Airport                   |41.1304722      |-80.6195833      |1044.0|-5 |A  |MAINLAND-EAST|AP  |
|06A|Moton Field Municipal Airport       |32.4605722      |-85.6800278      |264.0 |-5 |A  |MAINLAND-EAST|AP  |
|06C|Schaumburg Regional                 |41.9893408      |-88.1012428      |801.0 |-6 |A  |MAINLAND-EAST|NaN |
|06N|Randall Airport                     |41.431912       |-74.3915611      |523.0 |-5 |A  |MAINLAND-EAST|AP  |
|09J|Jekyll Island Airport               |31.0744722      |-81.4277778      |11.0  |-4 |A  |MAINLAND-EAST|AP  |
|0A9|Elizabethton Municipal Airport      |36.3712222      |-82.1734167      |1593.0|-4 |A  |MAINLAND-EAS

### Pergunta 6

In [22]:
MILITARY = ["Base", "Aaf", "AFs", "Ahp", "Afb", "LRRS", "Lrrs", "Arb", "Naf", "NAS", "Nas", "Jrb", "Ns", "As", "Cgas", "Angb"]

MILITARY_REGEX = r'|'.join(map(lambda x : f"(^{x} .*|.* {x} .*|.* {x}$)", MILITARY))

In [23]:
MILITARY_REGEX

'(^Base .*|.* Base .*|.* Base$)|(^Aaf .*|.* Aaf .*|.* Aaf$)|(^AFs .*|.* AFs .*|.* AFs$)|(^Ahp .*|.* Ahp .*|.* Ahp$)|(^Afb .*|.* Afb .*|.* Afb$)|(^LRRS .*|.* LRRS .*|.* LRRS$)|(^Lrrs .*|.* Lrrs .*|.* Lrrs$)|(^Arb .*|.* Arb .*|.* Arb$)|(^Naf .*|.* Naf .*|.* Naf$)|(^NAS .*|.* NAS .*|.* NAS$)|(^Nas .*|.* Nas .*|.* Nas$)|(^Jrb .*|.* Jrb .*|.* Jrb$)|(^Ns .*|.* Ns .*|.* Ns$)|(^As .*|.* As .*|.* As$)|(^Cgas .*|.* Cgas .*|.* Cgas$)|(^Angb .*|.* Angb .*|.* Angb$)'

In [24]:
airports = airports.withColumn('military',
                              F.when(F.col('name').rlike(MILITARY_REGEX), True)
                              .otherwise(False))

In [25]:
airports.filter(F.col('military') == True).toPandas()

,faa,name,lat,lon,alt,tz,dst,region,type,military
0,ADW,Andrews Afb,38.810806,-76.867028,280.0,-5,A,MAINLAND-EAST,NaN,True
1,AGN,Angoon Seaplane Base,57.503611,-134.585000,0.0,-9,A,ALASKA,NaN,True
2,ALZ,Alitak Seaplane Base,56.899444,-154.247778,0.0,-9,A,ALASKA,NaN,True
3,AOS,Amook Bay Seaplane Base,57.471389,-153.815278,0.0,-9,A,ALASKA,NaN,True
4,APG,Phillips Aaf,39.466219,-76.168808,57.0,-5,A,MAINLAND-EAST,NaN,True
...,...,...,...,...,...,...,...,...,...,...
155,WRI,Mc Guire Afb,40.015556,-74.591667,131.0,-5,A,MAINLAND-EAST,NaN,True
156,WSD,Condron Aaf,32.341484,-106.402770,3934.0,-7,A,MAINLAND-WEST,NaN,True
157,WSJ,San Juan - Uganik Seaplane Base,57.730278,-153.320556,0.0,-9,A,ALASKA,NaN,True
158,WSX,Westsound Seaplane Base,48.617778,-122.952778,0.0,-8,A,MAINLAND-WEST,NaN,True


### Pergunta 7

In [36]:
INTER = ["International", "Intl", "Intercontinental"]
INTER_REGEX = r'|'.join(map(lambda x : f".*({x}).*", INTER))
INTER_REGEX

'.*(International).*|.*(Intl).*|.*(Intercontinental).*'

In [35]:
REGIONAL = ["Regional", "Reigonal", "Rgnl", "County", "Metro", "Metropolitan"]
REGIONAL_REGEX = r'|'.join(map(lambda x: f".*({x}).*", REGIONAL))
REGIONAL_REGEX 

'.*(Regional).*|.*(Reigonal).*|.*(Rgnl).*|.*(County).*|.*(Metro).*|.*(Metropolitan).*'

In [33]:
MUNICIPAL = ["Municipal", "Muni", "City"]
MUNICIPAL_REGEX = r'|'.join(map(lambda x: f".*({x}).*", MUNICIPAL))
MUNICIPAL_REGEX

'.*(Municipal).*|.*(Muni).*|.*(City).*'

In [37]:
airports = airports.withColumn('administration',
                              F.when(F.col('name').rlike(INTER_REGEX), 'I')
                              .when(F.col('name').rlike(r'.*National.*|.*Natl.*'), 'N')
                              .when(F.col('name').rlike(REGIONAL_REGEX), 'R')
                              .when(F.col('name').rlike(MUNICIPAL_REGEX), 'M')
                              .otherwise(float('nan')))

In [38]:
airports.select('name', 'administration').show(truncate = False)

+------------------------------------+--------------+
|name                                |administration|
+------------------------------------+--------------+
|Lansdowne Airport                   |NaN           |
|Moton Field Municipal Airport       |M             |
|Schaumburg Regional                 |R             |
|Randall Airport                     |NaN           |
|Jekyll Island Airport               |NaN           |
|Elizabethton Municipal Airport      |M             |
|Williams County Airport             |R             |
|Finger Lakes Regional Airport       |R             |
|Shoestring Aviation Airfield        |NaN           |
|Jefferson County Intl               |I             |
|Harford County Airport              |R             |
|Galt Field Airport                  |NaN           |
|Port Bucyrus-Crawford County Airport|R             |
|Jackson County Airport              |R             |
|Martin Campbell Field Airport       |NaN           |
|Mansfield Municipal        

# Salvando em parquet

In [39]:
airports.write.parquet(
    mode ='overwrite',
    path = 'C:/Practice_Pyspark/airports_proc/parquet')


In [40]:
airports_parquet = spark.read.parquet('C:/Practice_Pyspark/airports_proc/parquet')

In [41]:
airports_parquet.toPandas()

,faa,name,lat,lon,alt,tz,dst,region,type,military,administration
0,04G,Lansdowne Airport,41.130472,-80.619583,1044.0,-5,A,MAINLAND-EAST,AP,False,NaN
1,06A,Moton Field Municipal Airport,32.460572,-85.680028,264.0,-5,A,MAINLAND-EAST,AP,False,M
2,06C,Schaumburg Regional,41.989341,-88.101243,801.0,-6,A,MAINLAND-EAST,NaN,False,R
3,06N,Randall Airport,41.431912,-74.391561,523.0,-5,A,MAINLAND-EAST,AP,False,NaN
4,09J,Jekyll Island Airport,31.074472,-81.427778,11.0,-4,A,MAINLAND-EAST,AP,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1392,ZUN,Black Rock,35.083228,-108.791778,6454.0,-7,A,MAINLAND-WEST,NaN,False,NaN
1393,ZVE,New Haven Rail Station,41.298669,-72.925992,7.0,-5,A,MAINLAND-EAST,AS,False,NaN
1394,ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0.0,-5,A,MAINLAND-EAST,AS,False,NaN
1395,ZWU,Washington Union Station,38.897460,-77.006430,76.0,-5,A,MAINLAND-EAST,AS,False,NaN


# Planes

In [42]:
schema_planes = StructType()\
    .add('tailnum', StringType(), True)\
    .add('year', IntegerType(), True)\
    .add('type', StringType(), True)\
    .add('manufacturer', StringType(), True)\
    .add('model', StringType(), True)\
    .add('engines', IntegerType(), True)\
    .add('seats', IntegerType(), True)\
    .add('speed', IntegerType(), True)\
    .add('engine', StringType(), True)

planes = spark.read.format('csv')\
    .option('header', True)\
    .schema(schema_planes)\
    .load('C:/Practice_Pyspark/planes.csv')

planes.printSchema()

root
 |-- tailnum: string (nullable = true)
 |-- year: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: integer (nullable = true)
 |-- engine: string (nullable = true)



### Pergunta 1

In [43]:
REGEX_TAILCHAR_2 = r'.*([ABCDEFGHJKLMNPQRSTUVWXYZ]{2}$)'
REGEX_TAILCHAR_1 = r'.*([ABCDEFGHJKLMNPQRSTUVWXYZ]$)'

In [44]:
planes = planes.withColumn('tailchar', 
                          F.when(
                              (F.length(F.col('tailnum')) == 6) & 
                              (F.col('tailnum').rlike(REGEX_TAILCHAR_2)), 
                              F.substring(F.col('tailnum'), 5, 2))
                          .when((F.length(F.col('tailnum')) == 6) & 
                               (F.col('tailnum').rlike(REGEX_TAILCHAR_1)), 
                               F.substring(F.col('tailnum'), 6, 1)))

In [45]:
planes.filter((F.col('tailchar').isNotNull())).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

### Pergunta 2

In [46]:
planes = planes.withColumn('year',
                          F.when(F.col('year') == 0, 1996)
                          .otherwise(F.col('year')))

In [47]:
planes.filter(F.col('year') == 1996).show(100)

+-------+----+--------------------+-----------------+-----------+-------+-----+-----+----------+--------+
|tailnum|year|                type|     manufacturer|      model|engines|seats|speed|    engine|tailchar|
+-------+----+--------------------+-----------------+-----------+-------+-----+-----+----------+--------+
| N188DN|1996|Fixed wing multi ...|           BOEING|    767-332|      2|  330| null| Turbo-fan|      DN|
| N234SW|1996|Fixed wing multi ...|          EMBRAER|  EMB-120ER|      2|   32| null|Turbo-prop|      SW|
| N235SW|1996|Fixed wing multi ...|          EMBRAER|  EMB-120ER|      2|   32| null|Turbo-prop|      SW|
| N237SW|1996|Fixed wing multi ...|          EMBRAER|  EMB-120ER|      2|   32| null|Turbo-prop|      SW|
| N290SW|1996|Fixed wing multi ...|          EMBRAER|  EMB-120ER|      2|   32| null|Turbo-prop|      SW|
| N291SW|1996|Fixed wing multi ...|          EMBRAER|  EMB-120ER|      2|   32| null|Turbo-prop|      SW|
| N292SW|1996|Fixed wing multi ...|          E

### Pergunta 3

In [48]:
from pyspark.sql.window import Window 

In [49]:
win_A = Window.partitionBy(F.col('manufacturer'), F.col('model')).orderBy(F.col('manufacturer'), F.col('model'), F.col('year'))

In [50]:
planes_s_Null = planes.filter(F.col('year').isNotNull())

In [51]:
pla_A = planes_s_Null.withColumn("row", F.row_number().over(win_A)) \
  .filter(F.col("row") == 1).drop("row")

In [52]:
pla_A.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N906FR|2002|Fixed wing multi ...|          AIRBUS|A319-111|      2|  147| null|Turbo-fan|      FR|
| N910FR|2002|Fixed wing multi ...|          AIRBUS|A319-112|      2|  100| null|Turbo-fan|      FR|
| N338NB|2002|Fixed wing multi ...|          AIRBUS|A319-114|      2|  145| null|Turbo-fan|      NB|
| N521VA|2006|Fixed wing multi ...|          AIRBUS|A319-115|      2|  147| null|Turbo-fan|      VA|
| N851UA|2002|Fixed wing multi ...|          AIRBUS|A319-131|      2|  179| null|Turbo-jet|      UA|
| N833AW|2002|Fixed wing multi ...|          AIRBUS|A319-132|      2|  179| null|Turbo-fan|      AW|
| N377NW|2003|Fixed wing multi ...|          AIRBUS|A320-211|      2|  182| null|Turbo-fan|

In [53]:
planes_A = planes.alias('ori').join(pla_A.alias('cop'), 
                                    (F.col('ori.model') == F.col('cop.model')) & 
                                    (F.col('ori.manufacturer') == F.col('cop.manufacturer')), 'inner')\
    .select(F.col('ori.tailnum'), F.col('ori.year'), F.col('ori.type'), F.col('ori.manufacturer'), F.col('ori.model'),\
           F.col('ori.engines'), F.col('ori.seats'), F.col('ori.speed'), F.col('ori.engine'), F.col('ori.tailchar'),\
           F.col('cop.year').alias('year_fill'))

In [54]:
planes_A.filter(F.col('year').isNull()).show()

+-------+----+--------------------+----------------+-----------+-------+-----+-----+---------+--------+---------+
|tailnum|year|                type|    manufacturer|      model|engines|seats|speed|   engine|tailchar|year_fill|
+-------+----+--------------------+----------------+-----------+-------+-----+-----+---------+--------+---------+
| N174US|null|Fixed wing multi ...|AIRBUS INDUSTRIE|   A321-211|      2|  199| null|Turbo-jet|      US|     2001|
| N177US|null|Fixed wing multi ...|AIRBUS INDUSTRIE|   A321-211|      2|  199| null|Turbo-jet|      US|     2001|
| N181UW|null|Fixed wing multi ...|AIRBUS INDUSTRIE|   A321-211|      2|  199| null|Turbo-jet|      UW|     2001|
| N194UW|null|Fixed wing multi ...|          AIRBUS|   A321-211|      2|  199| null|Turbo-fan|      UW|     2002|
| N271LV|null|Fixed wing multi ...|          BOEING|    737-705|      2|  149| null|Turbo-fan|      LV|     1998|
| N298WN|null|Fixed wing multi ...|          BOEING|    737-7H4|      2|  140| null|Turb

In [55]:
planes_A = planes_A.withColumn('year', 
                              F.coalesce(F.col('year'), F.col('year_fill'))).drop(F.col('year_fill'))

In [56]:
planes_A.show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|

### Pergunta 4

In [57]:
planes_A = planes_A.withColumn('age',
                          F.when(F.col('year').isNotNull(), 
                                 F.date_format(F.current_date(), 'yyyy') - F.col('year')).cast('int'))

In [58]:
planes_A.filter(F.col('age').isNotNull()).show()

+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A32

### Pergunta 5

In [59]:
TYPE = ["Fixed wing multi engine", "Fixed wing single engine", "Rotorcraft"]

In [60]:
planes_A = planes_A.withColumn('type',
                          F.when(F.col('type') == TYPE[0], 'MULTI_ENG')
                          .when(F.col('type') == TYPE[1], 'SINGLE_ENG')
                          .when(F.col('type') == TYPE[2], 'ROTORCRAFT'))

In [61]:
planes_A.filter(F.col('type') != 'MULTI_ENG').show()

+-------+----+----------+--------------------+----------+-------+-----+-----+-------------+--------+---+
|tailnum|year|      type|        manufacturer|     model|engines|seats|speed|       engine|tailchar|age|
+-------+----+----------+--------------------+----------+-------+-----+-----+-------------+--------+---+
| N201AA|1959|SINGLE_ENG|              CESSNA|       150|      1|    2|   90|Reciprocating|      AA| 63|
| N347AA|1985|ROTORCRAFT|            SIKORSKY|     S-76A|      2|   14| null|  Turbo-shaft|      AA| 37|
| N425AA|1968|SINGLE_ENG|               PIPER| PA-28-180|      1|    4|  107|Reciprocating|      AA| 54|
| N508AA|1975|ROTORCRAFT|                BELL|      206B|      1|    5|  112|  Turbo-shaft|      AA| 47|
| N508JB|2007|SINGLE_ENG|  CIRRUS DESIGN CORP|      SR22|      1|    4| null|Reciprocating|      JB| 15|
| N520AA|1985|SINGLE_ENG|        KILDALL GARY| FALCON-XP|      1|    2| null|Reciprocating|      AA| 37|
| N537JB|2012|ROTORCRAFT|ROBINSON HELICOPT...|       R6

### Pergunta 6

In [62]:
MANUFACTURER = ['AIRBUS', 'BOEING', 'BOMBARDIER', 'CESSNA', 'EMBRAER', 'SIKORSKY', 'CANADAIR', 'PIPER', 'MCDONNELL DOUGLAS',
                'CIRRUS', 'BELL', 'KILDALL GARY', 'LAMBERT RICHARD', 'BARKER JACK', 'ROBINSON HELICOPTER', 'GULFSTREAM'
                'MARZ BARRY']

print(len(MANUFACTURER))

16


In [63]:
planes_A = planes_A.withColumn('manufacturer',
                           F.when(F.col('manufacturer').contains(MANUFACTURER[0]), MANUFACTURER[0])
                          .when(F.col('manufacturer').contains(MANUFACTURER[1]), MANUFACTURER[1])
                          .when(F.col('manufacturer').contains(MANUFACTURER[2]), MANUFACTURER[2])
                          .when(F.col('manufacturer').contains(MANUFACTURER[3]), MANUFACTURER[3])
                          .when(F.col('manufacturer').contains(MANUFACTURER[4]), MANUFACTURER[4])
                          .when(F.col('manufacturer').contains(MANUFACTURER[5]), MANUFACTURER[5])
                          .when(F.col('manufacturer').contains(MANUFACTURER[6]), MANUFACTURER[6])
                          .when(F.col('manufacturer').contains(MANUFACTURER[7]), MANUFACTURER[7])
                          .when(F.col('manufacturer').contains(MANUFACTURER[8]), MANUFACTURER[8])
                          .when(F.col('manufacturer').contains(MANUFACTURER[9]), MANUFACTURER[9])
                          .when(F.col('manufacturer').contains(MANUFACTURER[10]), MANUFACTURER[10])
                          .when(F.col('manufacturer').contains(MANUFACTURER[11]), MANUFACTURER[11])
                          .when(F.col('manufacturer').contains(MANUFACTURER[12]), MANUFACTURER[12])
                          .when(F.col('manufacturer').contains(MANUFACTURER[13]), MANUFACTURER[13])
                          .when(F.col('manufacturer').contains(MANUFACTURER[14]), MANUFACTURER[14])
                          .when(F.col('manufacturer').contains(MANUFACTURER[15]), MANUFACTURER[15]))


In [64]:
planes_A.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N111US|1

### Pergunta 7

In [65]:
planes_A = planes_A.withColumn('model', 
                               F.regexp_replace('model', r'[(].*[)]', ''))

In [66]:
planes_A.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N111US|1

### Pergunta 8

In [67]:
planes_A.filter(F.col('speed').isNull()).show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182| null|Turbo-fan|      UW| 23|
| N111US|1

In [68]:
planes_A = planes_A.withColumn('speed',
                              F.when(F.col('speed').isNull(), F.ceil(F.col('seats') / 0.36).cast('int'))
                              .otherwise(F.col('speed')))

In [69]:
planes_A.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 24|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US| 23|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US| 23|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|
| N110UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|
| N111US|1

### Pergunta 9

In [70]:
ENGINE_TYPE = ["Turbo-fan", "Turbo-jet", "Turbo-prop", "Turbo-shaft", "4 Cycle"]

In [71]:
planes_A = planes_A.withColumn('engine_type',
                              F.when(F.col('engine') == ENGINE_TYPE[0], 'FAN')
                              .when(F.col('engine') == ENGINE_TYPE[1], 'JET')
                              .when(F.col('engine') == ENGINE_TYPE[2], 'PROP')
                              .when(F.col('engine') == ENGINE_TYPE[3], 'SHAFT')
                              .when(F.col('engine') == ENGINE_TYPE[4], 'CYCLE'))

In [72]:
planes_A.show()

+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+-----------+
|tailnum|year|     type|manufacturer|   model|engines|seats|speed|   engine|tailchar|age|engine_type|
+-------+----+---------+------------+--------+-------+-----+-----+---------+--------+---+-----------+
| N102UW|1998|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 24|        FAN|
| N103US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US| 23|        FAN|
| N104UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|        FAN|
| N105UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|        FAN|
| N107US|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      US| 23|        FAN|
| N108UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      UW| 23|        FAN|
| N109UW|1999|MULTI_ENG|      AIRBUS|A320-214|      2|  182|  506|Turbo-fan|      

In [73]:
planes_A.write.parquet(
    mode ='overwrite',
    path = 'C:/Practice_Pyspark/planes_proc/parquet')

# Flights

In [103]:
schema_flights = StructType()\
    .add('year', IntegerType(), True)\
    .add('month', IntegerType(), True)\
    .add('day', IntegerType(), True)\
    .add('dep_time', StringType(), True)\
    .add('dep_delay', StringType(), True)\
    .add('arr_time', StringType(), True)\
    .add('arr_delay', StringType(), True)\
    .add('carrier', StringType(), True)\
    .add('tailnum', StringType(), True)\
    .add('flight', StringType(), True)\
    .add('origin', StringType(), True)\
    .add('dest', StringType(), True)\
    .add('air_time', IntegerType(), True)\
    .add('distance', IntegerType(), True)\
    .add('hour', IntegerType(), True)\
    .add('minute', IntegerType(), True)

flights = spark.read.format('csv')\
    .option('header', True)\
    .schema(schema_flights)\
    .load('C:/Practice_Pyspark/flights.csv')

flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)



### Pergunta 1

In [104]:
flights = flights.withColumn('hour',
                            F.when(F.col('hour').isNull(), 0)
                            .otherwise(F.col('hour')))

In [105]:
flights = flights.withColumn('minute',
                            F.when(F.col('minute').isNull(), 0)
                            .otherwise(F.col('minute')))

In [106]:
flights.select('minute').distinct().show(60)

+------+
|minute|
+------+
|    31|
|    53|
|    34|
|    28|
|    26|
|    27|
|    44|
|    12|
|    22|
|    47|
|     1|
|    52|
|    13|
|    16|
|     6|
|     3|
|    40|
|    20|
|    57|
|    54|
|    48|
|     5|
|    19|
|    41|
|    43|
|    15|
|    37|
|     9|
|    17|
|    35|
|    55|
|     4|
|    59|
|     8|
|    23|
|    39|
|    49|
|     7|
|    51|
|    10|
|    50|
|    45|
|    38|
|    25|
|    24|
|    29|
|    21|
|    32|
|    56|
|    58|
|    11|
|    33|
|    14|
|    42|
|     2|
|    30|
|    46|
|     0|
|    18|
|    36|
+------+



### Pergunta 2

In [107]:
flights.filter(F.col('hour') == 24).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|    6| 20|    2400|       10|     535|        0|     AA| N3GHAA|  2486|   SEA| ORD|     197|    1721|  24|     0|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+



In [108]:
flights = flights.withColumn('hour',
                            F.when(F.col('hour') == 24, 0)
                            .otherwise(F.col('hour')))

In [109]:
flights.filter(F.col('hour') == 24).show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+



### Pergunta 3

In [110]:
def time_10(time):
    return F.col(time) < 10

def string_10(time):
    return F.concat(F.lit('0'), F.col(time).cast(StringType()))

In [111]:
flights = flights.withColumn('month_s',
                   F.when(time_10('month'), string_10('month'))
                  .otherwise(F.col('month').cast(StringType())))

flights.show()

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|month_s|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|     12|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|     01|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|     03|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|     04|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     9

In [112]:
flights = flights.withColumn('day_s',
                   F.when(time_10('day'), string_10('day'))
                  .otherwise(F.col('day').cast(StringType())))

In [113]:
flights = flights.withColumn('hour_s',
                   F.when(time_10('hour'), string_10('hour'))
                  .otherwise(F.col('hour').cast(StringType())))

In [114]:
flights = flights.withColumn('minute_s',
                   F.when(time_10('minute'), string_10('minute'))
                  .otherwise(F.col('minute').cast(StringType())))

In [92]:
flights.toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,month_s,day_s,hour_s,minute_s
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,6,58,12,08,06,58
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,10,40,01,22,10,40
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,14,43,03,09,14,43
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,17,5,04,09,17,05
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,7,54,03,09,07,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2014,6,23,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,18,6,06,23,18,06
9996,2014,8,31,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,23,36,08,31,23,36
9997,2014,8,8,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,9,4,08,08,09,04
9998,2014,8,29,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,14,41,08,29,14,41


In [115]:
flights = flights.withColumn('timestamp_s', 
                            F.concat(F.col('year').cast(StringType()), F.col('month_s'),
                                     F.col('day_s'), F.col('hour_s'), F.col('minute_s'), F.lit('00')))

In [116]:
flights.toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,...,dest,air_time,distance,hour,minute,month_s,day_s,hour_s,minute_s,timestamp_s
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,...,LAX,132.0,954,6,58,12,08,06,58,20141208065800
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,...,HNL,360.0,2677,10,40,01,22,10,40,20140122104000
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,...,SFO,111.0,679,14,43,03,09,14,43,20140309144300
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,...,SJC,83.0,569,17,5,04,09,17,05,20140409170500
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,...,BUR,127.0,937,7,54,03,09,07,54,20140309075400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2014,6,23,1806,-4,2104,-6,OO,N225AG,3458,...,SLC,89.0,689,18,6,06,23,18,06,20140623180600
9996,2014,8,31,2336,11,452,-13,AA,N3LEAA,1230,...,DFW,178.0,1660,23,36,08,31,23,36,20140831233600
9997,2014,8,8,904,-1,1042,-5,AS,N523AS,360,...,SMF,81.0,605,9,4,08,08,09,04,20140808090400
9998,2014,8,29,1441,26,1820,10,WN,N8647A,2857,...,ABQ,133.0,1180,14,41,08,29,14,41,20140829144100


In [117]:
flights = flights.withColumn('dep_datetime', 
                  F.to_timestamp('timestamp_s', 'yyyyMMddHHmmss'))\
.drop('month_s', 'day_s', 'hour_s', 'minute_s', 'timestamp_s')

In [118]:
flights.printSchema()

root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: string (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- minute: integer (nullable = true)
 |-- dep_datetime: timestamp (nullable = true)



In [23]:
flights.select('flight', 'dep_datetime').show()

+------+-------------------+
|flight|       dep_datetime|
+------+-------------------+
|  1780|2014-12-08 06:58:00|
|   851|2014-01-22 10:40:00|
|   755|2014-03-09 14:43:00|
|   344|2014-04-09 17:05:00|
|   522|2014-03-09 07:54:00|
|    48|2014-01-15 10:37:00|
|  1520|2014-07-02 08:47:00|
|   755|2014-05-12 16:55:00|
|   490|2014-04-19 12:36:00|
|    26|2014-11-19 18:12:00|
|   448|2014-11-08 16:53:00|
|   656|2014-08-03 11:20:00|
|   608|2014-10-30 08:11:00|
|   121|2014-11-12 23:46:00|
|   306|2014-10-31 13:14:00|
|  1458|2014-01-29 20:09:00|
|   368|2014-12-17 20:15:00|
|   827|2014-08-11 10:17:00|
|    24|2014-01-13 21:56:00|
|  3488|2014-06-05 17:33:00|
+------+-------------------+
only showing top 20 rows



In [ ]:
#f"{F.col('year')}-{F.col('month'):02d}-{F.col('day'):02d} {F.col('hour'):02d}:{F.col('minute'):02d}:00".strftime("....")

In [ ]:
#F.make_timestamp

### Pergunta 4

In [119]:
flights.filter(F.col('dep_time') == 'NA').show(50)

+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|       dep_datetime|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+-------------------+
|2014|    3|  4|      NA|       NA|      NA|       NA|     UA|     NA|   156|   SEA| DEN|    null|    1024|   0|     0|2014-03-04 00:00:00|
|2014|    2| 12|      NA|       NA|      NA|       NA|     AS| N527AS|     2|   SEA| DCA|    null|    2329|   0|     0|2014-02-12 00:00:00|
|2014|    7|  1|      NA|       NA|      NA|       NA|     WN| N8323C|  2485|   SEA| MDW|    null|    1733|   0|     0|2014-07-01 00:00:00|
|2014|    4| 30|      NA|       NA|      NA|       NA|     AS| N526AS|   566|   PDX| LAX|    null|     834|   0|     0|2014-04-30 00:00:00|
|2014|    1|  3|    

In [120]:
flights = flights.withColumn('dep_time',
                            F.when(F.col('dep_time') == 'NA', 
                                   F.concat(F.col('hour').cast(StringType()), 
                                            F.lpad(F.col('minute').cast(StringType()), 2, '0')))
                            .otherwise(F.col('dep_time')))

In [101]:
flights.toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,dep_datetime
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,6,58,2014-12-08 06:58:00
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,10,40,2014-01-22 10:40:00
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,14,43,2014-03-09 14:43:00
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,17,5,2014-04-09 17:05:00
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,7,54,2014-03-09 07:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2014,6,23,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,18,6,2014-06-23 18:06:00
9996,2014,8,31,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,23,36,2014-08-31 23:36:00
9997,2014,8,8,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,9,4,2014-08-08 09:04:00
9998,2014,8,29,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,14,41,2014-08-29 14:41:00


### Pergunta 5

In [121]:
flights.filter(F.col('dep_delay') == 'NA').toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,dep_datetime
0,2014,3,4,000,NA,NA,NA,UA,NA,156,SEA,DEN,NaN,1024,0,0,2014-03-04
1,2014,2,12,000,NA,NA,NA,AS,N527AS,2,SEA,DCA,NaN,2329,0,0,2014-02-12
2,2014,7,1,000,NA,NA,NA,WN,N8323C,2485,SEA,MDW,NaN,1733,0,0,2014-07-01
3,2014,4,30,000,NA,NA,NA,AS,N526AS,566,PDX,LAX,NaN,834,0,0,2014-04-30
4,2014,1,3,000,NA,NA,NA,US,NA,553,SEA,PHL,NaN,2378,0,0,2014-01-03
5,2014,8,7,000,NA,NA,NA,AS,N579AS,867,SEA,OGG,NaN,2640,0,0,2014-08-07
6,2014,8,11,000,NA,NA,NA,OO,N689CA,4528,PDX,SEA,NaN,129,0,0,2014-08-11
7,2014,1,2,000,NA,NA,NA,UA,NA,212,SEA,EWR,NaN,2402,0,0,2014-01-02
8,2014,5,15,000,NA,NA,NA,OO,N917SW,6250,PDX,LAX,NaN,834,0,0,2014-05-15
9,2014,2,7,000,NA,NA,NA,OO,N594SW,5553,PDX,SFO,NaN,550,0,0,2014-02-07


In [127]:
flights = flights.withColumn('dep_delay',
                            F.when(F.col('dep_delay') == 'NA', 0)
                            .otherwise(F.col('dep_delay')))

In [128]:
flights.toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,dep_datetime
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,6,58,2014-12-08 06:58:00
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,10,40,2014-01-22 10:40:00
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,14,43,2014-03-09 14:43:00
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,17,5,2014-04-09 17:05:00
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,7,54,2014-03-09 07:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2014,6,23,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,18,6,2014-06-23 18:06:00
9996,2014,8,31,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,23,36,2014-08-31 23:36:00
9997,2014,8,8,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,9,4,2014-08-08 09:04:00
9998,2014,8,29,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,14,41,2014-08-29 14:41:00


### Pergunta 6

In [129]:
flights.filter(F.col('arr_delay') == 'NA').toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,dep_datetime


In [126]:
flights = flights.withColumn('arr_delay',
                            F.when(F.col('arr_delay') == 'NA', 0)
                            .otherwise(F.col('arr_delay')))

In [35]:
flights.toPandas()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,dep_datetime
0,2014,12,8,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,6,58,2014-12-08 06:58:00
1,2014,1,22,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,10,40,2014-01-22 10:40:00
2,2014,3,9,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,14,43,2014-03-09 14:43:00
3,2014,4,9,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,17,5,2014-04-09 17:05:00
4,2014,3,9,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,7,54,2014-03-09 07:54:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,2014,6,23,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,18,6,2014-06-23 18:06:00
9996,2014,8,31,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,23,36,2014-08-31 23:36:00
9997,2014,8,8,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,9,4,2014-08-08 09:04:00
9998,2014,8,29,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,14,41,2014-08-29 14:41:00


### Pergunta 7

In [130]:
flights = flights.drop('year', 'month', 'day', 'hour', 'minute')

In [131]:
flights.show()

+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|       dep_datetime|
+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+-------------------+
|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|2014-12-08 06:58:00|
|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|2014-01-22 10:40:00|
|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|2014-03-09 14:43:00|
|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|2014-04-09 17:05:00|
|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|2014-03-09 07:54:00|
|    1037|        7|    1352|        2|     WN| N646SW|    48|   PDX| DEN|     121|     991|2014

### Pergunta 8

In [132]:
flights = flights.withColumn('air_time_projected',
                            F.round((F.col('distance') * 0.1 + 20), 0).cast('int'))

In [133]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,2014-12-08 06:58:00,115
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,2014-01-22 10:40:00,288
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,2014-03-09 14:43:00,88
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,2014-04-09 17:05:00,77
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,2014-03-09 07:54:00,114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,2014-06-23 18:06:00,89
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,2014-08-31 23:36:00,186
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,2014-08-08 09:04:00,81
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,2014-08-29 14:41:00,138


### Pergunta 9

In [134]:
flights_mean = flights.groupBy(F.col('dest'), F.col('origin'))\
    .agg(F.round(F.avg('air_time'), 0).cast('int').alias('air_time_expected'))

In [135]:
flights_mean.show()

+----+------+-----------------+
|dest|origin|air_time_expected|
+----+------+-----------------+
| ORD|   PDX|              206|
| HOU|   PDX|              226|
| IAD|   SEA|              264|
| CLE|   SEA|              234|
| BWI|   SEA|              270|
| DEN|   SEA|              127|
| RNO|   SEA|               74|
| PSP|   SEA|              129|
| STL|   PDX|              204|
| SJC|   PDX|               86|
| DFW|   SEA|              196|
| COS|   SEA|              133|
| SNA|   PDX|              121|
| PDX|   SEA|               33|
| LMT|   PDX|               54|
| MDW|   PDX|              214|
| MCI|   PDX|              174|
| EUG|   PDX|               27|
| DFW|   PDX|              192|
| SMF|   PDX|               68|
+----+------+-----------------+
only showing top 20 rows



In [136]:
flights = flights.alias('f1').join(flights_mean.alias('f2'),\
                         (F.col('f1.origin') == F.col('f2.origin')) &
                         (F.col('f1.dest') == F.col('f2.dest')), 'inner')\
                        .select('f1.dep_time', 
                                'f1.dep_delay', 
                                'f1.arr_time', 
                                'f1.arr_delay', 
                                'f1.carrier', 
                                'f1.tailnum',
                                'f1.flight', 
                                'f1.origin', 
                                'f1.dest', 
                                'f1.air_time', 
                                'f1.distance', 
                                'f1.dep_datetime',
                                'f1.air_time_projected', 
                                'f2.air_time_expected')

In [137]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,2014-12-08 06:58:00,115,127
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,2014-01-22 10:40:00,288,344
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,2014-03-09 14:43:00,88,101
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,2014-04-09 17:05:00,77,86
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,2014-03-09 07:54:00,114,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,2014-06-23 18:06:00,89,89
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,2014-08-31 23:36:00,186,196
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,2014-08-08 09:04:00,81,83
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,2014-08-29 14:41:00,138,143


### Pergunta 10

In [138]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,132.0,954,2014-12-08 06:58:00,115,127
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,360.0,2677,2014-01-22 10:40:00,288,344
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,111.0,679,2014-03-09 14:43:00,88,101
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,83.0,569,2014-04-09 17:05:00,77,86
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,127.0,937,2014-03-09 07:54:00,114,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89.0,689,2014-06-23 18:06:00,89,89
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,178.0,1660,2014-08-31 23:36:00,186,196
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,81.0,605,2014-08-08 09:04:00,81,83
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,133.0,1180,2014-08-29 14:41:00,138,143


In [139]:
flights = flights.withColumn('air_time',
                  F.greatest(F.col('air_time_projected'), F.col('air_time_expected')))

In [140]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,127,954,2014-12-08 06:58:00,115,127
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,344,2677,2014-01-22 10:40:00,288,344
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,101,679,2014-03-09 14:43:00,88,101
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,86,569,2014-04-09 17:05:00,77,86
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,122,937,2014-03-09 07:54:00,114,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,89,89
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,196,1660,2014-08-31 23:36:00,186,196
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,83,605,2014-08-08 09:04:00,81,83
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,143,1180,2014-08-29 14:41:00,138,143


### Pergunta 11

In [141]:
flights = flights.withColumn('dep_time_m',
                  F.when(F.length(F.col('dep_time')) == 3, 
                         (F.substring(F.col('dep_time'), 1, 1). cast('int') * 60) + 
                         F.substring(F.col('dep_time'), 2, 2).cast('int'))
                  .otherwise(
                      (F.substring(F.col('dep_time'), 1, 2).cast('int') * 60) + 
                      F.substring(F.col('dep_time'), 3, 2).cast('int')))

In [142]:
flights = flights.withColumn('arr_time_',
                  F.concat(F.floor((F.col('dep_time_m') + F.col('air_time')) / 60).cast(StringType()), 
                           F.lpad((F.col('dep_time_m') + F.col('air_time')) % 60, 2, '0').cast(StringType())))

In [143]:
flights = flights.withColumn('arr_time_',
                  F.when((F.length(F.col('arr_time_')) == 4) & 
                         (F.substring(F.col('arr_time_'), 1, 2).cast('int') > 23), 
                                F.concat((F.substring(F.col('arr_time_'), 1, 2).cast('int') - 24).cast(StringType()),
                                F.substring(F.col('arr_time_'), 3, 2)))
                  .otherwise(F.col('arr_time_'))).drop('dep_time_m')

In [144]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected,arr_time_
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,127,954,2014-12-08 06:58:00,115,127,905
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,344,2677,2014-01-22 10:40:00,288,344,1624
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,101,679,2014-03-09 14:43:00,88,101,1624
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,86,569,2014-04-09 17:05:00,77,86,1831
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,122,937,2014-03-09 07:54:00,114,122,956
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,89,89,1935
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,196,1660,2014-08-31 23:36:00,186,196,252
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,83,605,2014-08-08 09:04:00,81,83,1027
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,143,1180,2014-08-29 14:41:00,138,143,1704


### Pergunta 12

In [145]:
flights = flights.withColumn('haul_duration',
                            F.when(F.col('air_time').between(20, 180), 'SHORT-HAUL')
                            .when(F.col('air_time').between(180, 360), 'MEDIUM-HAUL')
                            .when(F.col('air_time') > 360, 'LONG-HAUL'))

In [146]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected,arr_time_,haul_duration
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,127,954,2014-12-08 06:58:00,115,127,905,SHORT-HAUL
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,344,2677,2014-01-22 10:40:00,288,344,1624,MEDIUM-HAUL
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,101,679,2014-03-09 14:43:00,88,101,1624,SHORT-HAUL
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,86,569,2014-04-09 17:05:00,77,86,1831,SHORT-HAUL
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,122,937,2014-03-09 07:54:00,114,122,956,SHORT-HAUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,89,89,1935,SHORT-HAUL
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,196,1660,2014-08-31 23:36:00,186,196,252,MEDIUM-HAUL
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,83,605,2014-08-08 09:04:00,81,83,1027,SHORT-HAUL
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,143,1180,2014-08-29 14:41:00,138,143,1704,SHORT-HAUL


### Pergunta 13

In [147]:
flights.orderBy('dep_datetime').toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected,arr_time_,haul_duration
0,000,0,NA,0,OO,N295SW,5403,PDX,EUG,31,106,2014-01-01 00:00:00,31,27,031,SHORT-HAUL
1,550,0,837,-12,DL,N660DL,1634,SEA,SLC,89,689,2014-01-01 05:50:00,89,89,719,SHORT-HAUL
2,600,-10,842,-8,AS,N786AS,426,SEA,LAX,127,954,2014-01-01 06:00:00,115,127,807,SHORT-HAUL
3,617,2,850,-12,AS,N579AS,494,SEA,SAN,138,1050,2014-01-01 06:17:00,125,138,835,SHORT-HAUL
4,619,-6,822,-7,VX,N841VA,751,SEA,SFO,101,679,2014-01-01 06:19:00,88,101,800,SHORT-HAUL
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1755,0,2149,10,US,N536UW,576,SEA,PHX,142,1107,2014-12-31 17:55:00,131,142,2017,SHORT-HAUL
9996,1755,0,1957,-7,AS,N403AS,302,SEA,SFO,101,679,2014-12-31 17:55:00,88,101,1936,SHORT-HAUL
9997,2054,-1,2326,7,AS,N519AS,534,SEA,ONT,125,956,2014-12-31 20:54:00,116,125,2259,SHORT-HAUL
9998,2120,-5,6,-9,AS,N768AS,121,SEA,ANC,191,1448,2014-12-31 21:20:00,165,191,031,MEDIUM-HAUL


In [148]:
flights = flights.withColumn('dep_season',
                  F.when(F.col('dep_datetime').between('2013-12-21 21:48', '2014-03-20 15:33') | 
                         F.col('dep_datetime').between('2014-12-21 21:48', '2015-03-20 15:33'), 'WINTER')
                  .when(F.col('dep_datetime').between('2014-03-20 15:33', '2014-06-21 10:14'), 'SPRING')
                  .when(F.col('dep_datetime').between('2014-06-21 10:14', '2014-09-23 02:04'), 'SUMMER')
                  .when(F.col('dep_datetime').between('2014-09-23 02:04', '2014-12-21 21:48'), 'FALL'))

In [149]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected,arr_time_,haul_duration,dep_season
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,127,954,2014-12-08 06:58:00,115,127,905,SHORT-HAUL,FALL
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,344,2677,2014-01-22 10:40:00,288,344,1624,MEDIUM-HAUL,WINTER
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,101,679,2014-03-09 14:43:00,88,101,1624,SHORT-HAUL,WINTER
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,86,569,2014-04-09 17:05:00,77,86,1831,SHORT-HAUL,SPRING
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,122,937,2014-03-09 07:54:00,114,122,956,SHORT-HAUL,WINTER
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,89,89,1935,SHORT-HAUL,SUMMER
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,196,1660,2014-08-31 23:36:00,186,196,252,MEDIUM-HAUL,SUMMER
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,83,605,2014-08-08 09:04:00,81,83,1027,SHORT-HAUL,SUMMER
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,143,1180,2014-08-29 14:41:00,138,143,1704,SHORT-HAUL,SUMMER


### Pergunta 14

In [150]:
flights = flights.withColumn('dep_delay_category',
                  F.when(F.col('dep_delay') < 0, 'ANTECIPATED')
                  .when(F.col('dep_delay') == 0, 'INTIME')
                  .when((F.col('dep_delay') > 0) & 
                        (F.col('dep_delay') < 60), 'MINOR')
                  .when(F.col('dep_delay') >= 60, 'MAJOR'))

In [151]:
flights.toPandas()

,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,dep_datetime,air_time_projected,air_time_expected,arr_time_,haul_duration,dep_season,dep_delay_category
0,658,-7,935,-5,VX,N846VA,1780,SEA,LAX,127,954,2014-12-08 06:58:00,115,127,905,SHORT-HAUL,FALL,ANTECIPATED
1,1040,5,1505,5,AS,N559AS,851,SEA,HNL,344,2677,2014-01-22 10:40:00,288,344,1624,MEDIUM-HAUL,WINTER,MINOR
2,1443,-2,1652,2,VX,N847VA,755,SEA,SFO,101,679,2014-03-09 14:43:00,88,101,1624,SHORT-HAUL,WINTER,ANTECIPATED
3,1705,45,1839,34,WN,N360SW,344,PDX,SJC,86,569,2014-04-09 17:05:00,77,86,1831,SHORT-HAUL,SPRING,MINOR
4,754,-1,1015,1,AS,N612AS,522,SEA,BUR,122,937,2014-03-09 07:54:00,114,122,956,SHORT-HAUL,WINTER,ANTECIPATED
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1806,-4,2104,-6,OO,N225AG,3458,SEA,SLC,89,689,2014-06-23 18:06:00,89,89,1935,SHORT-HAUL,SUMMER,ANTECIPATED
9996,2336,11,452,-13,AA,N3LEAA,1230,SEA,DFW,196,1660,2014-08-31 23:36:00,186,196,252,MEDIUM-HAUL,SUMMER,MINOR
9997,904,-1,1042,-5,AS,N523AS,360,SEA,SMF,83,605,2014-08-08 09:04:00,81,83,1027,SHORT-HAUL,SUMMER,ANTECIPATED
9998,1441,26,1820,10,WN,N8647A,2857,SEA,ABQ,143,1180,2014-08-29 14:41:00,138,143,1704,SHORT-HAUL,SUMMER,MINOR


In [152]:
flights.write.parquet(
    mode = 'overwrite',
    path = 'C:/Practice_Pyspark/flights_proc/parquet')